# 3D simulation and fitting

This tutorial shows how to do a 3D map-based simulation and fit.

For a tutorial on how to do a 3D map analyse of existing data, see the [analysis_3d](analysis_3d.ipynb) tutorial.

This can be useful to do a performance / sensitivity study, or to evaluate the capabilities of Gammapy or a given analysis method. Note that is is a binned simulation as is e.g. done also in Sherpa for Chandra, not an event sampling and anbinned analysis as is done e.g. in the Fermi ST or ctools.

## Imports and versions

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import astropy.units as u
from astropy.coordinates import Angle, SkyCoord
from gammapy.irf import load_cta_irfs
from gammapy.maps import WcsGeom, MapAxis
from gammapy.modeling.models import PowerLawSpectralModel
from gammapy.modeling.models import GaussianSpatialModel
from gammapy.modeling.models import SkyModel
from gammapy.cube import MapDataset, MapDatasetMaker, simulate_map
from gammapy.modeling import Fit
from gammapy.data import Observation

In [ ]:
!gammapy info --no-envvar --no-dependencies --no-system

## Simulation

We will simulate using the CTA-1DC IRFs shipped with gammapy. The main things to do are load the IRFs, define the observation parameters and the sky model to use for simulation. We then use `simualate_map()` to  create an in-memory observation and an empty dataset, predict the number of counts for the given model, and Poission fluctuate it to make a simulated counts maps.

In [ ]:
# Loading IRFs
irfs = load_cta_irfs(
    "$GAMMAPY_DATA/cta-1dc/caldb/data/cta/1dc/bcf/South_z20_50h/irf_file.fits"
)

In [ ]:
# Define the observation parameters (typically the observation duration and the pointing position):
livetime = 1.0 * u.hr
pointing = SkyCoord(0, 0, unit="deg", frame="galactic")

In [ ]:
# Define map geometry for binned simulation
energy_reco = MapAxis.from_edges(
    np.logspace(-1, 1.0, 10), unit="TeV", name="energy", interp="log"
)
geom = WcsGeom.create(
    skydir=(0, 0), binsz=0.02, width=(5, 4), coordsys="GAL", axes=[energy_reco]
)
print(energy_reco.edges)

In [ ]:
# Define sky model to used simulate the data.
# Here we use a Gaussian spatial model and a Power Law spectral model.
spatial_model = GaussianSpatialModel(
    lon_0="0.2 deg", lat_0="0.1 deg", sigma="0.3 deg", frame="galactic"
)
spectral_model = PowerLawSpectralModel(
    index=2.0, amplitude="1e-10 cm-2 s-1 TeV-1", reference="1 TeV"
)
model_simu = SkyModel(
    spatial_model=spatial_model, spectral_model=spectral_model
)
print(model_simu)

Now, comes the main part of dataset simulation. You can pass further arguments into `simulate_map` like the parameters of the background model and the binnings for the IRF maps.

In [ ]:
dataset = simulate_map(
    skymodel=model_simu,
    irfs=irfs,
    pointing=pointing,
    livetime=livetime,
    geom=geom,
)

Now use this dataset as you would in all standard analysis. You can plot the maps, or proceed with your custom analysis. 
In the next section, we show the standard 3D fitting as in [analysis_3d](analysis_3d.ipynb).

In [ ]:
# To plot, eg, counts:
dataset.counts.smooth(0.1 * u.deg).plot_interactive(add_cbar=True)

In [ ]:
print(dataset)

## Fit

In this section, we do a usual 3D fit with the same model used to simulated the data and see the stability of the simulations. Often, it is useful to simulate many such datasets and look at the distribution of the reconstructed parameters.

In [ ]:
# Make a copy of the dataset
dataset1 = dataset.copy()

In [ ]:
# Define sky model to fit the data
spatial_model1 = GaussianSpatialModel(
    lon_0="0.1 deg", lat_0="0.1 deg", sigma="0.5 deg", frame="galactic"
)
spectral_model1 = PowerLawSpectralModel(
    index=2, amplitude="1e-11 cm-2 s-1 TeV-1", reference="1 TeV"
)
model_fit = SkyModel(
    spatial_model=spatial_model1, spectral_model=spectral_model1
)

dataset1.model = model_fit
print(model_fit)

In [ ]:
# We do not want to fit the background in this case, so we will freeze the parameters
background_model = dataset1.background_model
background_model.parameters["norm"].value = 1.0
background_model.parameters["norm"].frozen = True
background_model.parameters["tilt"].frozen = True

print(background_model)

In [ ]:
%%time
fit = Fit([dataset1])
result = fit.run(optimize_opts={"print_level": 1})

Compare the injected and fitted models: 

In [ ]:
print("True model: \n", model_simu, "\n\n Fitted model: \n", model_fit)

Check the parameter table to see the best fit values along with the errors obtained from the covariance matrix

In [ ]:
result.parameters.to_table()